This notebook is used to analyze the molecular structures of the collected datasets.<br> 

In [1]:
import pandas as pd

from functools import partial
from collections import Counter

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect as Morgan

from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
import rdkit.Chem.Fragments as Fragments

In [2]:
import warnings
warnings.filterwarnings('ignore')

-  some functions from MOSES (https://github.com/molecularsets/moses)

In [3]:
def get_mol(smiles_or_mol):
    '''
    Loads SMILES/molecule into RDKit's object
    '''
    if isinstance(smiles_or_mol, str):
        if len(smiles_or_mol) == 0:
            return None
        mol = Chem.MolFromSmiles(smiles_or_mol)
        if mol is None:
            return None
        try:
            Chem.SanitizeMol(mol)
        except ValueError:
            return None
        return mol
    return smiles_or_mol

In [4]:
def mapper(n_jobs):
    '''
    Returns function for map call.
    If n_jobs == 1, will use standard map
    If n_jobs > 1, will use multiprocessing pool
    If n_jobs is a pool object, will return its map function
    '''
    if n_jobs == 1:
        def _mapper(*args, **kwargs):
            return list(map(*args, **kwargs))

        return _mapper
    if isinstance(n_jobs, int):
        pool = Pool(n_jobs)

        def _mapper(*args, **kwargs):
            try:
                result = pool.map(*args, **kwargs)
            finally:
                pool.terminate()
            return result

        return _mapper
    return n_jobs.map

In [5]:
def compute_scaffold(mol, min_rings=1):
    mol = get_mol(mol)
    try:
        scaffold = MurckoScaffold.GetScaffoldForMol(mol)
    except (ValueError, RuntimeError):
        return None
    n_rings = get_n_rings(scaffold)
    scaffold_smiles = Chem.MolToSmiles(scaffold)
    if scaffold_smiles == '' or n_rings < min_rings:
        return None
    return scaffold_smiles

In [6]:
def compute_scaffolds(mol_list, n_jobs=1, min_rings=1):
    """
    Extracts a scafold from a molecule in a form of a canonic SMILES
    """
    scaffolds = Counter()
    map_ = mapper(n_jobs)
    scaffolds = Counter(
        map_(partial(compute_scaffold, min_rings=min_rings), mol_list))
    if None in scaffolds:
        scaffolds.pop(None)
    return scaffolds

-  some other functions to extract the strutural traits

In [7]:
def get_weight(mol):
    """
    Computes molecular weight for given molecule.
    Returns float,
    """
    return Descriptors.MolWt(mol)

In [8]:
def get_n_rings(mol):
    """
    Computes the number of rings in a molecule
    """
    return mol.GetRingInfo().NumRings()

In [9]:
def get_scaffold(mol):
    mol = get_mol(mol)
    scaffold = MurckoScaffold.GetScaffoldForMol(mol)
    scaffold_smiles = Chem.MolToSmiles(scaffold)
    return scaffold_smiles

In [10]:
def get_n_atoms(mol):
    """
    Computes the number of atoms in a molecule
    """
    return rdMolDescriptors.CalcNumAtoms(mol)

In [11]:
def get_n_heavy_atoms(mol):
    """
    Computes the number of heavy atoms in a molecule
    """
    return rdMolDescriptors.CalcNumHeavyAtoms(mol)

In [12]:
def get_n_hetero_atoms(mol):
    """
    Computes the number of hetero atoms in a molecule
    """
    return rdMolDescriptors.CalcNumHeteroatoms(mol)

In [13]:
def get_f_sp3C(mol):
    """
    Computes the fraction of sp3 carbon in a molecule
    """
    return rdMolDescriptors.CalcFractionCSP3(mol)

In [14]:
def get_n_aroma_rings(mol):
    """
    Computes the number of aromatic rings in a molecule
    """
    return rdMolDescriptors.CalcNumAromaticRings(mol)

In [15]:
def get_n_alipha_rings(mol):
    """
    Computes the number of aliphatic rings in a molecule
    """
    return rdMolDescriptors.CalcNumAliphaticRings(mol)

In [16]:
def get_n_hba(mol):
    """
    Computes the number of h-bond acceptors in a molecule
    """
    return rdMolDescriptors.CalcNumHBA(mol)

In [17]:
def get_n_hbd(mol):
    """
    Computes the number of h-bond donors in a molecule
    """
    return rdMolDescriptors.CalcNumHBD(mol)

In [18]:
def get_n_rota_bonds(mol):
    """
    Computes the number of rotatable bonds in a molecule
    """
    return rdMolDescriptors.CalcNumRotatableBonds(mol)

In [19]:
def get_n_stereo_centers(mol):
    """
    Computes the number of stereo centers in a molecule
    """
    try:
        return rdMolDescriptors.CalcNumAtomStereoCenters(mol)
    except ValueError:
        return 0

In [20]:
def get_n_hetero_rings(mol):
    """
    Computes the number of heterocycles in a molecule
    """
    return rdMolDescriptors.CalcNumHeterocycles(mol)

In [21]:
  def get_n_spiro_atoms(mol):
    """
    Computes the number of heterocycles in a molecule
    """
    return rdMolDescriptors.CalcNumSpiroAtoms(mol)

# Calculate scaffold frequency

-  Bemis-Murcko scaffolds with # minimum ring set as 1 (https://pubs.acs.org/doi/10.1021/jm9602928)

In [22]:
#opioids: MDR1, CYP2D6, CYP3A4 | MOR, DOR, KOR
#benchmark: BACE, BBBP, HIV | ESOL, FreeSolv, Lipop
folder = 'opioids'
mol_prop = 'KOR' 
task_setting = 'reg'# benchmark: benchmark; opioids: reg

#read the csv file
df = pd.read_csv('../data/{folder}/{name}_{task}.csv'.format(folder=folder, name=mol_prop, task=task_setting))
print("df.shape", df.shape)

#get the Bemis-Murcko scaffolds dict from the dataset
scaffolds_dict = dict(compute_scaffolds(list(df['SMILES'])))

#convert the scaffolds dict to a df
scaffolds_df = pd.DataFrame.from_dict(scaffolds_dict, orient='index')
scaffolds_df.reset_index(inplace=True)
scaffolds_df.columns = ['SMILES', 'freq']

# sort the scaffolds by frequency in descending order and add rank
scaffolds_df = scaffolds_df.sort_values(by="freq", ascending=False).reset_index(drop=True)
scaffolds_df['rank'] = scaffolds_df.index + 1
print(scaffolds_df.shape)

df.shape (3326, 2)
(1660, 3)


In [23]:
scaffolds_df.head()

,SMILES,freq,rank
0,c1ccc(C2CCN(C(c3ccccc3)c3ccccc3)CC2)cc1,59,1
1,O=C1CCCC2CC[C@H]3C(=O)O[C@H](c4ccoc4)CC3[C@@H]12,47,2
2,O=c1cnc2ccccc2n1C1CCN(C2CCCCCCC2)CC1,34,3
3,c1cc2c3c(c1)O[C@H]1C4CCC5(CC4)[C@@H](C2)N(CC2C...,34,4
4,O=c1c(-c2ccccc2)cccn1C(CN1CCCC1)c1ccccc1,33,5


In [24]:
# save the scaffolds_df.csv 
scaffolds_df.to_csv('../results/structures/{folder}/{mol_prop}_scaffolds_freq.csv'.format(folder=folder, mol_prop=mol_prop), index=False)

# Extract structural traits 

-  for each molecule in the dataset, we extract the following structural traits: <br> 
BM-scaffolds, fragments & other structural traits (e.g. MW, n_rings)

In [25]:
# fragments are taken from rdkit.Chem.Fragments module
RDKit_fragments = ['fr_Al_COO', 'fr_Al_OH', 'fr_Al_OH_noTert', 'fr_ArN', 'fr_Ar_COO', 'fr_Ar_N', 'fr_Ar_NH', 'fr_Ar_OH',
'fr_COO', 'fr_COO2', 'fr_C_O', 'fr_C_O_noCOO', 'fr_C_S', 'fr_HOCCN', 'fr_Imine', 'fr_NH0', 'fr_NH1', 'fr_NH2', 'fr_N_O',
'fr_Ndealkylation1', 'fr_Ndealkylation2', 'fr_Nhpyrrole', 'fr_SH', 'fr_aldehyde', 'fr_alkyl_carbamate', 'fr_alkyl_halide',
'fr_allylic_oxid', 'fr_amide', 'fr_amidine', 'fr_aniline', 'fr_aryl_methyl', 'fr_azide', 'fr_azo', 'fr_barbitur',
'fr_benzene', 'fr_benzodiazepine', 'fr_bicyclic', 'fr_diazo', 'fr_dihydropyridine', 'fr_epoxide', 'fr_ester', 'fr_ether',
'fr_furan', 'fr_guanido', 'fr_halogen', 'fr_hdrzine', 'fr_hdrzone', 'fr_imidazole', 'fr_imide', 'fr_isocyan', 'fr_isothiocyan',
'fr_ketone', 'fr_ketone_Topliss', 'fr_lactam', 'fr_lactone', 'fr_methoxy', 'fr_morpholine', 'fr_nitrile', 'fr_nitro',
'fr_nitro_arom', 'fr_nitro_arom_nonortho', 'fr_nitroso', 'fr_oxazole', 'fr_oxime', 'fr_para_hydroxylation', 'fr_phenol',
'fr_phenol_noOrthoHbond', 'fr_phos_acid', 'fr_phos_ester', 'fr_piperdine', 'fr_piperzine', 'fr_priamide', 'fr_prisulfonamd',
'fr_pyridine', 'fr_quatN', 'fr_sulfide', 'fr_sulfonamd', 'fr_sulfone', 'fr_term_acetylene', 'fr_tetrazole', 'fr_thiazole',
'fr_thiocyan', 'fr_thiophene', 'fr_unbrch_alkane', 'fr_urea']

In [26]:
# specify dataset details
folder = "opioids"
# benchmark for benchmark; reg for opioids
task_setting = "reg"  

#opioids: 'MDR1', 'CYP2D6', 'CYP3A4', 'MOR', 'DOR', 'KOR' 
#benchmark: 'BACE', 'BBBP', 'HIV', 'ESOL', 'FreeSolv', 'Lipop'
mol_props =  ['MDR1', 'CYP2D6', 'CYP3A4', 'MOR', 'DOR', 'KOR' ]

In [27]:
for mol_prop in mol_props:
    print("mol_prop", mol_prop)
    
    #read the csv file
    df = pd.read_csv('../data/{folder}/{name}_{task}.csv'.format(folder=folder, name=mol_prop, task=task_setting))

    #add scaffolds 
    df['scaffold'] = df['SMILES'].map(get_scaffold)
    
    # add fragments
    for frag in RDKit_fragments:
        fragment_analysis = """
#define a function for fragments counting    
def count_fragments(mol):
    return Fragments.{frag}(mol)

#count the number of fragments for each molecule
df['{frag}'] = df['SMILES'].map(get_mol).map(count_fragments) 
        """.format(frag=frag)
        #execute the text as code
        exec(fragment_analysis)
    
    # add other structural traits
    df['MW'] = df['SMILES'].map(get_mol).map(get_weight)
    df['f_sp3C'] = df['SMILES'].map(get_mol).map(get_f_sp3C)
    df['n_hetero_atoms'] = df['SMILES'].map(get_mol).map(get_n_hetero_atoms)
    df['n_stereo_centers'] = df['SMILES'].map(get_mol).map(get_n_stereo_centers)
    df['n_hba'] = df['SMILES'].map(get_mol).map(get_n_hba)
    df['n_hbd'] = df['SMILES'].map(get_mol).map(get_n_hbd)
    df['n_rota_bonds'] = df['SMILES'].map(get_mol).map(get_n_rota_bonds)
    df['n_rings'] = df['SMILES'].map(get_mol).map(get_n_rings)
    df['n_alipha_rings'] = df['SMILES'].map(get_mol).map(get_n_alipha_rings)
    df['n_aroma_rings'] = df['SMILES'].map(get_mol).map(get_n_aroma_rings)
    df['n_hetero_rings'] = df['SMILES'].map(get_mol).map(get_n_hetero_rings)
    df['n_spiro_atoms'] = df['SMILES'].map(get_mol).map(get_n_spiro_atoms)

    #save the structural traits csv file
    df.to_csv('../results/structures/{folder}/{mol_prop}_structural_traits.csv'.format(folder=folder, mol_prop=mol_prop), index=False)

mol_prop MDR1
mol_prop CYP2D6
mol_prop CYP3A4
mol_prop MOR
mol_prop DOR
mol_prop KOR
